# Ejercicio 2: 

Escribe un programa en Python que use Biopython para hacer una búsqueda BLAST de una secuencia de proteína que introduzcas por teclado.
El programa debe guardar en un fichero los resultados que tengan un E-value menor que 0.001. 
El fichero debe contener el identificador, la longitud, el E-value y el porcentaje de identidad de cada resultado. 
Hágalo de forma online y local.

In [43]:
import os
import warnings
import time
import pandas as pd
import json

In [37]:
warnings.filterwarnings("ignore")

from debug_sequence.functions import check_protein
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastpCommandline
from Bio.Blast import NCBIXML

### 2.1. Instalación de BLAST

1. Se descarga la versión para el equipo de BLAST desde el siguiente enlace: [BLAST](https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/)
2. Se ejecuta, en nuestro caso, el *.exe* descargado y se instala en el equipo.
3. Se configura el *path* de BLAST como variable de entorno del sistema.
4. Se verifica la instalación con el comando `blastn --version`.

### 2.2. Secuencia de aminoácidos por teclado

In [33]:
def get_protein():
    protein = input("Enter the protein sequence: ")
    protein = check_protein(protein)
    return protein

### 2.3. Implementación online

Enlace a la documentación: [BLAST Record](https://biopython.org/docs/1.75/api/Bio.Blast.NCBIWWW.html)

Los atributos indicados en el enunciado son:
- Identificador: `alignment.hit_id`
- Longitud: `alignment.length`
- E-value: `hsp.expect`
- Porcentaje de identidad: `hsp.identities / hsp.align_length` (número de identidades / longitud de la secuencia de consulta)


In [47]:
def search_protein_sequence(output_file="./results/blast_results.txt"):

    protein_sequence = get_protein()

    result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence)
    blast_records = NCBIXML.parse(result_handle)

    filtered_results = []

    for blast_record in blast_records:
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                if hsp.expect < 0.001:
                    result = {
                        "id": alignment.hit_id,
                        "length": alignment.length,
                        "evalue": hsp.expect,
                        "identity_percentage": (hsp.identities / hsp.align_length) * 100
                    }
                    filtered_results.append(result)

    with open(output_file, "w") as f:
        json.dump(filtered_results, f, indent=4)

    print(f"Resultados guardados en formato JSON en: {output_file}")


### 2.4. Implementación local

Para usar las bases de datos proporcionadas por el NCBI, es necesario descargarlas mediante el comando `perl update_blastdb.pl --decompress bbdd`, donde `bbdd` es la base de datos de secuencias de aminoácidos. Las bases de datos utilizadas son las siguientes:

- `swissprot`: repositorio de secuencias de proteínas que forma parte de la base de datos UniProt.
- `nr`: base de datos no redundante de secuencias de proteínas. Debido a las limitaciones de recursos, se han descargado únicamente los archivos desde el 000 al 020.



In [45]:
def search_local_protein(output_file = "./results/blast_results_locally.txt"):
    os.environ["CMD"] = input("Introduce la ruta del ejecutable de BLAST: ")
    os.environ["BLASTDB"] = input("Introduce la ruta de la base de datos de BLAST: ")
    sequence = input("Introduce la secuencia de proteína en formato FASTA: ")

    fasta_file = "query.fasta"
    with open(fasta_file, "w") as f:
        f.write(">query\n")
        f.write(sequence)

    blastp_cline = NcbiblastpCommandline(
        cmd=os.environ["CMD"],
        query=fasta_file,
        db=os.environ["BLASTDB"],
        evalue=0.001,
        outfmt=5,
        out="results.xml"
    )

    blastp_cline()

    with open("results.xml") as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        filtered_results = []

        for blast_record in blast_records:
            for alignment in blast_record.alignments:
                for hsp in alignment.hsps:
                    if hsp.expect < 0.001:
                        result = {
                            "id": alignment.hit_id,
                            "length": alignment.length,
                            "evalue": hsp.expect,
                            "identity_percentage": (hsp.identities / hsp.align_length) * 100
                        }
                        filtered_results.append(result)

    with open(output_file, "w") as f:
        json.dump(filtered_results, f, indent=4)

    print(f"Resultados guardados en formato JSON en: {output_file}")

    os.remove(fasta_file)
    os.remove("results.xml")

### 2.5. Comparación de la implementación local vs online

Al igual que el ejercicio 1, se ha decidido realizar una comparativa entre la implementación local y online. Para ello, se utilizará parte de la secuencia de aminoácidos correspondiente a la proteína de la insulina humana.

...`LVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGP`...

**CMD** 

C:\Users\Susana\Desktop\blast\blast-2.16.0+\bin\blastp.exe

**DB**

C:\Users\Susana\Desktop\blast\blast-2.16.0+\db\swissprot\swissprot

C:\Users\Susana\Desktop\blast\blast-2.16.0+\db\nr\nr


In [48]:
import time

times_execution = {}

#### 2.5.1. Implementación online

In [49]:
start = time.time()
search_protein_sequence()
end = time.time()

times_execution["online"] = end - start

Resultados guardados en formato JSON en: ./results/blast_results.txt


#### 2.5.2. Implementación local - `swissprot`

In [50]:
start = time.time()
search_local_protein()
end = time.time()

times_execution["local swissprot"] = end - start

Resultados guardados en formato JSON en: ./results/blast_results_locally.txt


#### 2.5.3. Implementación local - subconjunto de `nr`

In [51]:
start = time.time()
search_local_protein(output_file="./results/blast_results_locally_nr.txt")
end = time.time()

times_execution["local nr subset"] = end - start

Resultados guardados en formato JSON en: ./results/blast_results_locally_nr.txt


#### 2.5.4. Comparación de resultados

En este caso, lo único que se puede comparar es el tiempo de ejecución, ya que los resultados más fiables se consideran los correspondientes a la base de datos online puesto que es la que más información contiene.

In [53]:
df = pd.DataFrame.from_dict(times_execution, orient="index", columns=["Tiempo de ejecución (s)"])
display(df)

,Tiempo de ejecución (s)
online,487.201359
local swissprot,14.008230
local nr subset,485.818624
